## WEEK-1 DETAILS:

### Methodology used:

In this notebook, I have focused on the corona virus cases worldwide and scrapped data from a number of websites:

#### 1) Worldometers corona virus page
       https://www.worldometers.info/coronavirus/
       From this website it collects the data on total cases,actice cases,deaths,recovered & total tests
#### 2) Worldometers population page
       https://www.worldometers.info/world-population/population-by-country/
       From this website it collects the data on the population in each country,population density & urban population percentage
#### 3) GDP of all the countries
    https://worldpopulationreview.com/countries/countries-by-gdp/
    From this website we directly download the GDP data csv file
#### 4) Tourism Data from Worldbank website
    https://data.worldbank.org/indicator/ST.INT.ARVL
    From this website we get the information on the number of tourists visiting any country in a year. From this website too, we directly download the csv

### Approach:
    1) Scrapping the corona virus data from source 1 into df1
    2) Scrapping the population data from source 2 into df2 and exporting it to csv
    3) Now preprocessing that csv to match the country names in df1
    4) Preprocessing the GDP csv to match the country names in df1
    5) Preprocessing the Tourism csv to match the country names in df1
    6) Importing the population,gdp and tourism csv again
    7) Merging the population,GDP & tourism dataframe to the coronavirus dataframe to get final df

### Conclusion:
   So by also adding the population data to the covid data, we can do in depth anaysis on the effect
   of population, density,urbanisation on the pandemic.
   
   GDP data would help us to analyse to what extent the bigger economies have been successful in containing the pandemic
   
   Lastly the Tourism in each country gives us the most important information about number of people travelling to that country in a year as It is the mojor reason for the spread of COVID-19

First importing all the libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

##### Getting the coronavirus data

In [2]:
url1 = "https://www.worldometers.info/coronavirus/"
response1 = requests.get(url1)
print (response1)

<Response [200]>


In [3]:
soup1 = BeautifulSoup(response1.content, "html.parser")
soup1.title

<title>Coronavirus Update (Live): 3,327,239 Cases and 234,702 Deaths from COVID-19 Virus Pandemic - Worldometer</title>

In [4]:
tables1 = soup1.find_all("table")
len(tables1)

2

In [5]:
coronatable = tables1[0]

In [6]:
country1 = []
totalcases = []
totaldeaths = []
totalrecovered = []
activecases = []
totaltests = []
rows = coronatable.find_all("tr")[9:-8]
colname = [country1,totalcases,totaldeaths,totalrecovered,activecases,totaltests]
colnum = [0,1,3,5,6,10]

for row in rows:
    col = row.find_all("td")
    for i,j in zip(colname,colnum):
        s = col[j].text.strip()
        s1 = s.replace(",","")
        i.append(s1)

In [10]:
df1 = pd.DataFrame(list(zip(country1,totalcases,totaldeaths,totalrecovered,activecases,totaltests)), columns=["Country","Total_Cases","Total_Deaths","Total_Recovered","Active_Cases","Total_Tests"])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country          214 non-null    object
 1   Total_Cases      214 non-null    object
 2   Total_Deaths     214 non-null    object
 3   Total_Recovered  214 non-null    object
 4   Active_Cases     214 non-null    object
 5   Total_Tests      214 non-null    object
dtypes: object(6)
memory usage: 10.2+ KB


In [11]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.replace(np.nan,0, inplace=True)
df.replace("N/A",0,inplace = True)

In [12]:
df.Total_Cases = pd.to_numeric(df.Total_Cases)
df.Total_Deaths = pd.to_numeric(df.Total_Deaths)
df.Total_Recovered = pd.to_numeric(df.Total_Recovered)
df.Active_Cases = pd.to_numeric(df.Active_Cases)
df.Total_Tests = pd.to_numeric(df.Total_Tests)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country          214 non-null    object
 1   Total_Cases      214 non-null    int64 
 2   Total_Deaths     214 non-null    int64 
 3   Total_Recovered  214 non-null    int64 
 4   Active_Cases     214 non-null    int64 
 5   Total_Tests      214 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 10.2+ KB


In [22]:
corona = df.sort_values("Country",ignore_index=True)
corona.head()

,Country,Total_Cases,Total_Deaths,Total_Recovered,Active_Cases,Total_Tests
0,Afghanistan,2335,68,310,1957,10593
1,Albania,773,31,470,272,8253
2,Algeria,4006,450,1779,1777,6500
3,Andorra,745,42,468,235,1673
4,Angola,27,2,7,18,0


##### Getting the population data

In [17]:
url2 = "https://www.worldometers.info/world-population/population-by-country/"
response2 = requests.get(url2)
print(response2)

<Response [200]>


In [18]:
soup2 = BeautifulSoup(response2.content, "html.parser")
soup2.title

<title>Population by Country (2020) - Worldometer</title>

In [19]:
tables2 = soup2.find_all("table")
len(tables2)

1

In [20]:
poptable = tables2[0]

In [21]:
country2 = []
population = []
density = []
urban = []
rows = poptable.find_all("tr")[1:]
colname = [population,density,urban]
colnum = [2,5,10]
for row in rows:
    col = row.find_all("td")
    country2.append(col[1].text.strip())
    for i,j in zip(colname,colnum):
        s = col[j].text.strip()
        s1 = s.replace(",","")
        s2 = s1.replace(" %","")
        s3 = s2.replace("N.A.","0")
        s4 = int(s3)
        i.append(s4)

In [24]:
df2 = pd.DataFrame(list(zip(country2,population,density,urban)), columns=['Country','Population','Population_Density','Urban_Population_%'])
df2.head()

,Country,Population,Population_Density,Urban_Population_%
0,China,1439323776,153,61
1,India,1380004385,464,35
2,United States,331002651,36,83
3,Indonesia,273523615,151,56
4,Pakistan,220892340,287,35


In [26]:
df3 = df2.sort_values("Country",ignore_index=True)
df3.head()

,Country,Population,Population_Density,Urban_Population_%
0,Afghanistan,38928346,60,25
1,Albania,2877797,105,63
2,Algeria,43851044,18,73
3,American Samoa,55191,276,88
4,Andorra,77265,164,88


##### Many countries are represented with different names/abbreviations so to avoid duplicates, I am exporting both the data as csv and comparing both of them manually to fix that

In [ ]:
corona.to_csv('1_corona_data.csv',index=False)

In [ ]:
df3.to_csv("2_Population_data.csv",index=False)

##### After processing the population data to match the country names of the covid data, importing it again (You can find the processed csv in the repository )

In [37]:
population = pd.read_csv("2_Population_data.csv")
population.head()

,Country,Population,Population_Density,Urban_Population_%
0,Afghanistan,38928346,60,25
1,Albania,2877797,105,63
2,Algeria,43851044,18,73
3,Andorra,77265,164,88
4,Angola,32866272,26,67


In [28]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country             214 non-null    object
 1   Population          214 non-null    int64 
 2   Population_Density  214 non-null    int64 
 3   Urban_Population_%  214 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.8+ KB


In [29]:
corona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country          214 non-null    object
 1   Total_Cases      214 non-null    int64 
 2   Total_Deaths     214 non-null    int64 
 3   Total_Recovered  214 non-null    int64 
 4   Active_Cases     214 non-null    int64 
 5   Total_Tests      214 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 10.2+ KB


##### Merging both to get a new dataframe result1

In [35]:
result1 = pd.merge(corona,population, on='Country')
result1.head()

,Country,Total_Cases,Total_Deaths,Total_Recovered,Active_Cases,Total_Tests,Population,Population_Density,Urban_Population_%
0,Afghanistan,2335,68,310,1957,10593,38928346,60,25
1,Albania,773,31,470,272,8253,2877797,105,63
2,Algeria,4006,450,1779,1777,6500,43851044,18,73
3,Andorra,745,42,468,235,1673,77265,164,88
4,Angola,27,2,7,18,0,32866272,26,67


In [36]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 0 to 213
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country             214 non-null    object
 1   Total_Cases         214 non-null    int64 
 2   Total_Deaths        214 non-null    int64 
 3   Total_Recovered     214 non-null    int64 
 4   Active_Cases        214 non-null    int64 
 5   Total_Tests         214 non-null    int64 
 6   Population          214 non-null    int64 
 7   Population_Density  214 non-null    int64 
 8   Urban_Population_%  214 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 16.7+ KB


##### Importing the GDP and the Tourism data csv

##### I have already filtered and processed the GDP and Travel data csv which you can find in the repository. If you want the raw csv then you can go to the links 3 and 4 mentioned at the start of this notebook

##### The GDP data shows the GDP of the countries in Million USD

In [41]:
gdp = pd.read_csv("3_GDP_data.csv")
gdp.head()

,Country,GDP_in_Mn
0,Afghanistan,20682
1,Albania,17210
2,Algeria,193056
3,Andorra,0
4,Angola,96426


##### The travel data shows the number of arrivals in thousands for a country in a year

In [43]:
travel = pd.read_csv("4_Travel_data.csv")

In [44]:
travel.head()

,Country,Num_of_Tourists_Arrival_in_1000s
0,Afghanistan,0
1,Albania,5340000
2,Algeria,2657000
3,Andorra,3042000
4,Angola,218000


Merging the GDP data to result1 

In [46]:
result2 = pd.merge(result1,gdp, on='Country')
result2.head()

,Country,Total_Cases,Total_Deaths,Total_Recovered,Active_Cases,Total_Tests,Population,Population_Density,Urban_Population_%,GDP_in_Mn
0,Afghanistan,2335,68,310,1957,10593,38928346,60,25,20682
1,Albania,773,31,470,272,8253,2877797,105,63,17210
2,Algeria,4006,450,1779,1777,6500,43851044,18,73,193056
3,Andorra,745,42,468,235,1673,77265,164,88,0
4,Angola,27,2,7,18,0,32866272,26,67,96426


##### Finally merging the travel data to get the final dataframe

In [48]:
final = pd.merge(result2,travel, on='Country')
final

,Country,Total_Cases,Total_Deaths,Total_Recovered,Active_Cases,Total_Tests,Population,Population_Density,Urban_Population_%,GDP_in_Mn,Num_of_Tourists_Arrival_in_1000s
0,Afghanistan,2335,68,310,1957,10593,38928346,60,25,20682,0
1,Albania,773,31,470,272,8253,2877797,105,63,17210,5340000
2,Algeria,4006,450,1779,1777,6500,43851044,18,73,193056,2657000
3,Andorra,745,42,468,235,1673,77265,164,88,0,3042000
4,Angola,27,2,7,18,0,32866272,26,67,96426,218000
...,...,...,...,...,...,...,...,...,...,...,...
209,Vietnam,270,0,219,51,261004,97338579,314,0,282372,15498000
210,Western Sahara,6,0,5,1,0,597339,2,87,0,381000
211,Yemen,6,2,1,3,120,29825964,56,38,31385,0
212,Zambia,106,3,55,48,5284,18383955,25,45,25272,1072000


In [50]:
final.to_csv("final.csv")

### Thank You!
### - Rahul Gupta